# Performance Evaluation Module Demo

This notebook demonstrates the usage of the **PerformanceEvaluator** class from the Plutus framework.

The PerformanceEvaluator provides 20+ industry-standard metrics for evaluating trading algorithm performance, including:
- **Return metrics**: Sharpe, Sortino, Calmar, Omega, Information Ratio, CAGR
- **Risk metrics**: VaR, CVaR, Volatility, Downside Deviation
- **Drawdown metrics**: Maximum, Average, Duration analysis

## Features
- Lazy computation with caching for optimal performance
- Decimal precision for financial calculations
- Backward compatible with existing code
- Easy-to-use API

In [ ]:
# Setup: Add src to path if running from examples directory
import sys
from pathlib import Path

# Add parent's src to path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root / 'src'))

from decimal import Decimal
from plutus.evaluation import PerformanceEvaluator
import random

print("✅ Imports successful!")

## Scenario 1: Basic Usage with Simple Returns

Let's start with a simple example showing daily returns from a trading strategy.

In [ ]:
# Simple daily returns (12 days of trading)
simple_returns = [
    Decimal('0.012'),   # Day 1: +1.2%
    Decimal('0.008'),   # Day 2: +0.8%
    Decimal('-0.005'),  # Day 3: -0.5%
    Decimal('0.015'),   # Day 4: +1.5%
    Decimal('-0.008'),  # Day 5: -0.8%
    Decimal('0.020'),   # Day 6: +2.0%
    Decimal('0.010'),   # Day 7: +1.0%
    Decimal('-0.012'),  # Day 8: -1.2%
    Decimal('0.018'),   # Day 9: +1.8%
    Decimal('0.005'),   # Day 10: +0.5%
    Decimal('-0.003'),  # Day 11: -0.3%
    Decimal('0.014'),   # Day 12: +1.4%
]

# Create evaluator using factory method (preferred)
evaluator = PerformanceEvaluator.from_returns(
    returns=simple_returns,
    annualization_factor=252,  # Daily returns
    risk_free_rate=Decimal('0.03'),  # 3% annual risk-free rate
    min_acceptable_return=Decimal('0.07')  # 7% MAR for Sortino
)

print("✅ PerformanceEvaluator created successfully!")
print(f"   Analyzing {len(simple_returns)} periods of returns")

### Core Metrics

All metrics are computed lazily and cached for performance.

In [ ]:
print("=" * 60)
print("CORE METRICS (Original 6)")
print("=" * 60)

print(f"Sharpe Ratio:              {evaluator.sharpe_ratio:.4f}")
print(f"Sortino Ratio:             {evaluator.sortino_ratio:.4f}")
print(f"Maximum Drawdown:          {evaluator.maximum_drawdown:.4f} ({evaluator.maximum_drawdown * 100:.2f}%)")
print(f"Annual Return:             {evaluator.annual_return:.4f} ({evaluator.annual_return * 100:.2f}%)")
print(f"Longest Drawdown Period:   {evaluator.longest_drawdown_period} periods")
print(f"")
print(f"Return Mean:               {evaluator.return_mean:.6f}")
print(f"Return Std Dev:            {evaluator.return_std:.6f}")

### Return-Based Metrics

In [ ]:
print("=" * 60)
print("RETURN METRICS (Phase 2)")
print("=" * 60)

print(f"Calmar Ratio:              {evaluator.calmar_ratio:.4f}")
print(f"Omega Ratio:               {evaluator.omega_ratio:.4f}")
print(f"CAGR:                      {evaluator.cagr:.4f} ({evaluator.cagr * 100:.2f}%)")
print(f"Total Return:              {evaluator.total_return:.4f} ({evaluator.total_return * 100:.2f}%)")

print("\nInterpretation:")
print(f"  • Calmar > 1.0 is good, > 3.0 is excellent")
print(f"  • Omega > 1.0 means strategy outperforms threshold")
print(f"  • CAGR is the annualized growth rate")
print(f"  • Total Return is the cumulative return over the period")

### Risk Metrics

In [ ]:
print("=" * 60)
print("RISK METRICS (Phase 2)")
print("=" * 60)

print(f"Value at Risk (95%):       {evaluator.value_at_risk_95:.4f} ({evaluator.value_at_risk_95 * 100:.2f}%)")
print(f"Value at Risk (99%):       {evaluator.value_at_risk_99:.4f} ({evaluator.value_at_risk_99 * 100:.2f}%)")
print(f"Conditional VaR (95%):     {evaluator.conditional_var_95:.4f} ({evaluator.conditional_var_95 * 100:.2f}%)")
print(f"Conditional VaR (99%):     {evaluator.conditional_var_99:.4f} ({evaluator.conditional_var_99 * 100:.2f}%)")
print(f"Annualized Volatility:     {evaluator.volatility:.4f} ({evaluator.volatility * 100:.2f}%)")
print(f"Downside Deviation:        {evaluator.downside_deviation:.4f} ({evaluator.downside_deviation * 100:.2f}%)")

print("\nInterpretation:")
print(f"  • VaR 95%: 95% confidence max loss won't exceed {abs(evaluator.value_at_risk_95 * 100):.2f}%")
print(f"  • CVaR: Expected loss in worst 5% of cases")
print(f"  • Volatility: Typical stocks have 15-30% annual volatility")

### Drawdown Metrics

In [ ]:
print("=" * 60)
print("DRAWDOWN METRICS (Phase 2)")
print("=" * 60)

print(f"Maximum Drawdown:          {evaluator.maximum_drawdown:.4f} ({evaluator.maximum_drawdown * 100:.2f}%)")
print(f"Average Drawdown:          {evaluator.average_drawdown:.4f} ({evaluator.average_drawdown * 100:.2f}%)")
print(f"Avg Drawdown Duration:     {evaluator.average_drawdown_duration:.2f} periods")
print(f"Longest DD Duration:       {evaluator.longest_drawdown_period} periods")

print("\nInterpretation:")
print(f"  • Max DD is the worst peak-to-trough decline")
print(f"  • Average DD shows typical decline from peaks")
print(f"  • DD Duration indicates how long recoveries take")

## Scenario 2: Comparing Multiple Strategies

Let's compare three different trading strategies with different risk/return profiles.

In [ ]:
# Generate synthetic returns for 3 strategies (60 trading days)
random.seed(42)  # For reproducibility

def generate_returns(mean_daily, volatility, num_days):
    """Generate synthetic daily returns."""
    returns = []
    for _ in range(num_days):
        # Simple normal distribution
        ret = random.gauss(mean_daily, volatility)
        returns.append(Decimal(str(round(ret, 6))))
    return returns

# Strategy 1: Aggressive (high return, high volatility)
aggressive_returns = generate_returns(
    mean_daily=0.0015,  # 0.15% daily avg
    volatility=0.025,   # 2.5% daily volatility
    num_days=60
)

# Strategy 2: Moderate (medium return, medium volatility)
moderate_returns = generate_returns(
    mean_daily=0.0008,  # 0.08% daily avg
    volatility=0.015,   # 1.5% daily volatility
    num_days=60
)

# Strategy 3: Conservative (low return, low volatility)
conservative_returns = generate_returns(
    mean_daily=0.0004,  # 0.04% daily avg
    volatility=0.008,   # 0.8% daily volatility
    num_days=60
)

# Create evaluators
eval_aggressive = PerformanceEvaluator.from_returns(aggressive_returns, annualization_factor=252)
eval_moderate = PerformanceEvaluator.from_returns(moderate_returns, annualization_factor=252)
eval_conservative = PerformanceEvaluator.from_returns(conservative_returns, annualization_factor=252)

print("✅ Created evaluators for 3 strategies")
print(f"   Each with {len(aggressive_returns)} days of returns")

In [ ]:
# Compare strategies
print("=" * 80)
print("STRATEGY COMPARISON")
print("=" * 80)
print(f"{'Metric':<25} {'Aggressive':>15} {'Moderate':>15} {'Conservative':>15}")
print("-" * 80)

metrics = [
    ('Total Return', 'total_return', 100),
    ('CAGR', 'cagr', 100),
    ('Sharpe Ratio', 'sharpe_ratio', 1),
    ('Sortino Ratio', 'sortino_ratio', 1),
    ('Calmar Ratio', 'calmar_ratio', 1),
    ('Omega Ratio', 'omega_ratio', 1),
    ('Max Drawdown', 'maximum_drawdown', 100),
    ('Avg Drawdown', 'average_drawdown', 100),
    ('Volatility', 'volatility', 100),
    ('VaR 95%', 'value_at_risk_95', 100),
    ('CVaR 95%', 'conditional_var_95', 100),
]

for metric_name, attr_name, multiplier in metrics:
    agg_val = getattr(eval_aggressive, attr_name) * multiplier
    mod_val = getattr(eval_moderate, attr_name) * multiplier
    con_val = getattr(eval_conservative, attr_name) * multiplier
    
    # Format based on metric type
    if multiplier == 100:  # Percentage
        print(f"{metric_name:<25} {agg_val:>14.2f}% {mod_val:>14.2f}% {con_val:>14.2f}%")
    else:  # Ratio
        print(f"{metric_name:<25} {agg_val:>15.4f} {mod_val:>15.4f} {con_val:>15.4f}")

print("\n📊 Analysis:")
print("  • Aggressive: Higher returns but more volatile and larger drawdowns")
print("  • Moderate: Balanced risk/return profile")
print("  • Conservative: Lower returns but more stable performance")

## Scenario 3: Real-World Pattern - Bull Market with Crash

Simulate a realistic scenario: steady bull market followed by a sharp crash and recovery.

In [ ]:
# Simulate a realistic market scenario
bull_market_returns = []

# Phase 1: Bull market (30 days) - steady gains with low volatility
for _ in range(30):
    ret = random.gauss(0.008, 0.005)  # 0.8% avg, 0.5% volatility
    bull_market_returns.append(Decimal(str(round(ret, 6))))

# Phase 2: Crash (5 days) - sharp decline
for _ in range(5):
    ret = random.gauss(-0.035, 0.015)  # -3.5% avg, high volatility
    bull_market_returns.append(Decimal(str(round(ret, 6))))

# Phase 3: Recovery (25 days) - gradual recovery
for _ in range(25):
    ret = random.gauss(0.012, 0.008)  # 1.2% avg, moderate volatility
    bull_market_returns.append(Decimal(str(round(ret, 6))))

eval_realistic = PerformanceEvaluator.from_returns(
    bull_market_returns,
    annualization_factor=252
)

print(f"✅ Created realistic market scenario: {len(bull_market_returns)} days")
print(f"   Phase 1: Bull market (30 days)")
print(f"   Phase 2: Crash (5 days)")
print(f"   Phase 3: Recovery (25 days)")

In [ ]:
print("=" * 60)
print("REALISTIC SCENARIO ANALYSIS")
print("=" * 60)

print("\n📈 RETURN METRICS")
print(f"Total Return:              {eval_realistic.total_return * 100:.2f}%")
print(f"CAGR:                      {eval_realistic.cagr * 100:.2f}%")
print(f"Annual Return:             {eval_realistic.annual_return * 100:.2f}%")

print("\n📊 RISK-ADJUSTED RETURNS")
print(f"Sharpe Ratio:              {eval_realistic.sharpe_ratio:.4f}")
print(f"Sortino Ratio:             {eval_realistic.sortino_ratio:.4f}")
print(f"Calmar Ratio:              {eval_realistic.calmar_ratio:.4f}")
print(f"Omega Ratio:               {eval_realistic.omega_ratio:.4f}")

print("\n⚠️  RISK METRICS")
print(f"Volatility:                {eval_realistic.volatility * 100:.2f}%")
print(f"Downside Deviation:        {eval_realistic.downside_deviation * 100:.2f}%")
print(f"VaR 95%:                   {eval_realistic.value_at_risk_95 * 100:.2f}%")
print(f"VaR 99%:                   {eval_realistic.value_at_risk_99 * 100:.2f}%")
print(f"CVaR 95%:                  {eval_realistic.conditional_var_95 * 100:.2f}%")
print(f"CVaR 99%:                  {eval_realistic.conditional_var_99 * 100:.2f}%")

print("\n📉 DRAWDOWN ANALYSIS")
print(f"Maximum Drawdown:          {eval_realistic.maximum_drawdown * 100:.2f}%")
print(f"Average Drawdown:          {eval_realistic.average_drawdown * 100:.2f}%")
print(f"Longest DD Duration:       {eval_realistic.longest_drawdown_period} days")
print(f"Avg DD Duration:           {eval_realistic.average_drawdown_duration:.2f} days")

print("\n💡 INSIGHTS")
print(f"  • Despite the crash, overall returns are positive")
print(f"  • Max drawdown captures the crash impact")
print(f"  • VaR/CVaR metrics quantify tail risk")
print(f"  • Calmar ratio shows return per unit of max drawdown risk")

## Scenario 4: Caching and Performance

Demonstrate the lazy computation and caching mechanism.

In [ ]:
import time

# Create evaluator with larger dataset
large_returns = generate_returns(0.001, 0.02, 1000)  # 1000 days
eval_large = PerformanceEvaluator.from_returns(large_returns, annualization_factor=252)

print("=" * 60)
print("CACHING DEMONSTRATION")
print("=" * 60)
print(f"Dataset size: {len(large_returns)} periods\n")

# First access - computation happens
start = time.time()
sharpe_1 = eval_large.sharpe_ratio
time_1 = (time.time() - start) * 1000
print(f"First access (compute):    {time_1:.3f} ms")

# Second access - cached value returned
start = time.time()
sharpe_2 = eval_large.sharpe_ratio
time_2 = (time.time() - start) * 1000
print(f"Second access (cached):    {time_2:.3f} ms")

print(f"\nSpeedup: {time_1 / time_2:.1f}x faster")
print(f"Same object: {sharpe_1 is sharpe_2}")

# Show cache contents
print(f"\nCached metrics: {list(eval_large._cache.keys())}")

# Clear cache
eval_large.clear_cache()
print(f"After clear_cache(): {list(eval_large._cache.keys())}")

## Scenario 5: Backward Compatibility

Show that the old `HistoricalPerformance` API still works.

In [ ]:
from plutus.evaluation import HistoricalPerformance

# Old API still works
old_style = HistoricalPerformance(
    returns=simple_returns,
    annualized_factor=Decimal('252'),
    risk_free_return=Decimal('0.03'),
    minimal_acceptable_return=Decimal('0.07')
)

print("=" * 60)
print("BACKWARD COMPATIBILITY TEST")
print("=" * 60)

print("\n✅ Old API (HistoricalPerformance) still works:")
print(f"Sharpe Ratio:              {old_style.sharpe_ratio:.4f}")
print(f"Sortino Ratio:             {old_style.sortino_ratio:.4f}")
print(f"Maximum Drawdown:          {old_style.maximum_drawdown:.4f}")
print(f"Annual Return:             {old_style.annual_return:.4f}")

print("\n✅ New metrics also available:")
print(f"Calmar Ratio:              {old_style.calmar_ratio:.4f}")
print(f"Omega Ratio:               {old_style.omega_ratio:.4f}")
print(f"VaR 95%:                   {old_style.value_at_risk_95 * 100:.2f}%")

print("\n💡 HistoricalPerformance is an alias to PerformanceEvaluator")
print(f"   Same class: {HistoricalPerformance is PerformanceEvaluator}")

## Scenario 6: Edge Cases

Test how the evaluator handles edge cases.

In [ ]:
print("=" * 60)
print("EDGE CASES")
print("=" * 60)

# All positive returns
all_positive = [Decimal('0.01'), Decimal('0.02'), Decimal('0.03')]
eval_positive = PerformanceEvaluator.from_returns(all_positive, annualization_factor=252)

print("\n1️⃣  All Positive Returns")
print(f"   Max Drawdown:           {eval_positive.maximum_drawdown:.6f} (should be 0)")
print(f"   Total Return:           {eval_positive.total_return * 100:.2f}%")
print(f"   Omega Ratio:            {eval_positive.omega_ratio:.4f} (should be very high)")

# All negative returns
all_negative = [Decimal('-0.01'), Decimal('-0.02'), Decimal('-0.03')]
eval_negative = PerformanceEvaluator.from_returns(all_negative, annualization_factor=252)

print("\n2️⃣  All Negative Returns")
print(f"   Max Drawdown:           {eval_negative.maximum_drawdown * 100:.2f}%")
print(f"   Total Return:           {eval_negative.total_return * 100:.2f}%")
print(f"   Omega Ratio:            {eval_negative.omega_ratio:.4f} (should be 0)")

# All zero returns
all_zeros = [Decimal('0')] * 10
eval_zeros = PerformanceEvaluator.from_returns(all_zeros, annualization_factor=252)

print("\n3️⃣  All Zero Returns")
print(f"   Sharpe Ratio:           {eval_zeros.sharpe_ratio:.4f} (should be 0)")
print(f"   Total Return:           {eval_zeros.total_return:.4f}")
print(f"   Max Drawdown:           {eval_zeros.maximum_drawdown:.4f}")

# Very high volatility
high_vol = [
    Decimal('0.10'), Decimal('-0.15'), Decimal('0.20'), 
    Decimal('-0.12'), Decimal('0.08')
]
eval_high_vol = PerformanceEvaluator.from_returns(high_vol, annualization_factor=252)

print("\n4️⃣  High Volatility Returns")
print(f"   Volatility:             {eval_high_vol.volatility * 100:.2f}%")
print(f"   Sharpe Ratio:           {eval_high_vol.sharpe_ratio:.4f}")
print(f"   Max Drawdown:           {eval_high_vol.maximum_drawdown * 100:.2f}%")
print(f"   VaR 95%:                {eval_high_vol.value_at_risk_95 * 100:.2f}%")

## Summary

### Key Takeaways

**Return Metrics (7):**
- Sharpe Ratio
- Sortino Ratio
- Calmar Ratio
- Omega Ratio
- Information Ratio
- CAGR
- Total Return

**Risk Metrics (6):**
- Value at Risk (95%, 99%)
- Conditional VaR (95%, 99%)
- Annualized Volatility
- Downside Deviation

**Drawdown Metrics (4):**
- Maximum Drawdown
- Average Drawdown
- Average Drawdown Duration
- Longest Drawdown Duration

**Basic Stats (5):**
- Return Mean
- Return Std Dev
- Annual Return
- Cumulative Performances
- Longest Drawdown Period

### Next Steps

- Use `PerformanceEvaluator` to analyze your trading strategies
- Compare multiple strategies to find the best risk/return profile
- Monitor performance metrics during backtesting and live trading

For more information, see the [Performance Evaluation Documentation and Guide](../src/plutus/evaluation/docs/).